In [ ]:
import imaplib
import email
import os
import getpass
!pip install SpeechRecognition pydub
import speech_recognition as sr
from pydub import AudioSegment

# ---------------------------
# 📌 1️⃣ Set Up FFmpeg Path
# ---------------------------
FFMPEG_PATH = r"C:\Users\saumy\ffmpeg-7.1.1-essentials_build\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe"
os.environ["PATH"] += os.pathsep + os.path.dirname(FFMPEG_PATH)
AudioSegment.converter = FFMPEG_PATH  # Set FFmpeg for pydub

# ---------------------------
# 📌 2️⃣ User Input for Email Login
# ---------------------------
EMAIL_HOST = "imap.gmail.com"
EMAIL_USER = input("📧 Enter your Gmail address: ").strip().replace("\xa0", "")
EMAIL_PASS = getpass.getpass("🔑 Enter your password: ").strip().replace("\xa0", "")

# ---------------------------
# 📌 3️⃣ Connect to Gmail
# ---------------------------
try:
    mail = imaplib.IMAP4_SSL(EMAIL_HOST)
    mail.login(EMAIL_USER, EMAIL_PASS)
    print("✅ Successfully logged in!")

    # Select the inbox
    mail.select("inbox")

    # Search for all emails
    status, email_ids = mail.search(None, "ALL")
    if status != "OK" or not email_ids[0]:
        print("❌ No emails found!")
        exit()

    email_ids = email_ids[0].split()
    print(f"📩 Found {len(email_ids)} emails.")

except imaplib.IMAP4.error:
    print("❌ Login failed! Check your email or password.")
    exit()

# ---------------------------
# 📌 4️⃣ Process Emails & Extract Audio
# ---------------------------
def save_audio_from_email(email_message):
    """Extracts and saves audio attachments from an email."""
    for part in email_message.walk():
        if part.get_content_maintype() == "multipart":
            continue
        if part.get("Content-Disposition") is None:
            continue

        filename = part.get_filename()
        if filename and filename.endswith((".mp3", ".wav", ".m4a")):
            os.makedirs("audio_files", exist_ok=True)  # Ensure folder exists
            filepath = os.path.join("audio_files", filename)
            with open(filepath, "wb") as f:
                f.write(part.get_payload(decode=True))
            print(f"🎵 Saved audio: {filename}")
            return filepath
    return None

audio_path = None  # Ensure it's always defined

# Process latest 5 emails
for num in email_ids[-5:]:
    status, data = mail.fetch(num, "(RFC822)")
    if status != "OK":
        print("❌ Failed to fetch email.")
        continue

    raw_email = data[0][1]
    msg = email.message_from_bytes(raw_email)

    # Extract and save audio
    audio_path = save_audio_from_email(msg)
    if audio_path:
        break  # Stop after finding the first audio file

if not audio_path:
    print("❌ No audio file found in the last 5 emails.")
    exit()

# ---------------------------
# 📌 5️⃣ Convert Audio to Text
# ---------------------------
def convert_audio_to_text(audio_path):
    """Converts an audio file to text using SpeechRecognition."""
    recognizer = sr.Recognizer()
    text = None  # Ensure text is always defined

    try:
        if not audio_path.endswith(".wav"):
            audio = AudioSegment.from_file(audio_path)
            new_audio_path = audio_path.rsplit(".", 1)[0] + ".wav"
            audio.export(new_audio_path, format="wav")
            audio_path = new_audio_path

        with sr.AudioFile(audio_path) as source:
            print("🎙️ Processing audio...")
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            print(f"📝 Transcribed Text: {text}")

    except sr.UnknownValueError:
        print("🤖 Could not understand the audio.")
    except sr.RequestError:
        print("⚠️ Speech Recognition API unavailable.")
    except Exception as e:
        print(f"❌ Error processing audio: {e}")

    return text

text = convert_audio_to_text(audio_path)

if not text:
    print("❌ No text extracted from audio.")
    exit()

# ---------------------------
# 📌 6️⃣ Spam Detection
# ---------------------------
SPAM_KEYWORDS = ["wow", "double", "lottery", "money", "details", "dump"]

def check_for_spam(text):
    """Checks if the text contains spam keywords."""
    words = text.lower().split()
    for word in words:
        if word in SPAM_KEYWORDS:
            print("🚨 SPAM DETECTED! Email moved to spam folder.")
            return True
    print("✅ No spam detected.")
    return False

is_spam = check_for_spam(text)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 57.0 MB/s eta 0:00:00
